In [1]:
import (
		"github.com/pa-m/sklearn/metrics"
	    ms "github.com/pa-m/sklearn/model_selection"
		"github.com/e-XpertSolutions/go-iforest/iforest"	
		"bufio"
		"encoding/csv"
		"strconv"
		"github.com/janpfeifer/gonb/gonbui"
		"gonum.org/v1/plot/plotutil"
		"gonum.org/v1/plot"
		"gonum.org/v1/plot/plotter"
		"gonum.org/v1/plot/vg"
		"gonum.org/v1/plot/vg/draw"
		"os/exec"
		"github.com/janpfeifer/gonb/cache"
		"math"
		"math/rand"

	)

	

# UTILITIES

In [2]:
type Split struct{
	TrainData [][] float64
	ValidateData [][] float64
	LabelData []int
	//trainLabelData []int for unit testing
	}

type config struct{
	treeNum int
	subsamplesize int
}

//load file Data and Return [][]float64 array
func loadData(myfile string) [][]float64 {

	l:=false
file, err := os.Open(myfile)
	if err != nil {
		fmt.Println("Error opening file:", err)
		return nil
	}
	defer file.Close()

	reader := csv.NewReader(file)
	records, err := reader.ReadAll()
	if err != nil {
		fmt.Println("Error reading CSV:", err)
		return nil
	}
	
	inputData := make([][]float64, len(records))
	for i, row := range records {

		inputData[i] = make([]float64, len(row))
		for j, value := range row {
			inputData[i][j], err = strconv.ParseFloat(value, 64)
			if err != nil {
				fmt.Printf("Error converting to float: line %d",i)
				l=true
				break	
			}
		}
		if l == true{
			break
		}
	}
	return inputData
}

func LoadLabels(filePath string)[]int{
	file, err := os.Open(filePath)
	if err != nil {
		fmt.Println("Error opening file:", err)
		return nil
	}
	defer file.Close()

	scanner := bufio.NewScanner(file)
	
	var labels []int

	for scanner.Scan() {
		label,err := strconv.Atoi(scanner.Text())
		if err != nil {
			// Handle the error if the conversion fails
			fmt.Println("Error:", err)
			return nil
		}
		labels= append(labels,label)
	}
	return labels
}

//Convert labels to float for F1 and Accuracy Score metrics
func ConvertToFloat(Labels_Pred []int , labels_Tru []int)([]float64 ,[]float64){

	length :=len(Labels_Pred)
	labelsPred_float := make([]float64, length)
	labelsTru_float := make([]float64, length)

	for i:=0; i<length; i++ {
		
		labelsPred_float[i] = float64(Labels_Pred[i])
		labelsTru_float[i] = float64(labels_Tru[i])
		
	}
	return labelsPred_float,labelsTru_float

}

func F1Score(labelsPred []int, labelsTru []int) float64{

	pred,tru := ConvertToFloat(labelsPred,labelsTru)

	Ypred, Ytrue := mat.NewDense(len(pred), 1, pred), mat.NewDense(len(tru), 1, tru)
	var sampleWeight []float64
	/* fmt.Printf("F1 macro %.2f\n", metrics.F1Score(Ytrue, Ypred, "macro", sampleWeight))
	fmt.Printf("F1 micro %.2f\n", metrics.F1Score(Ytrue, Ypred, "micro", sampleWeight))
	fmt.Printf("F1 weighted %.2f\n", metrics.F1Score(Ytrue, Ypred, "weighted", sampleWeight)) */
	return metrics.F1Score(Ytrue, Ypred, "weighted", sampleWeight)

}

func AccuracyScore(labelsPred []int, labelsTru []int)float64{

	pred,tru := ConvertToFloat(labelsPred,labelsTru)

	var nilDense *mat.Dense
	normalize, sampleWeight := true, nilDense
	Ypred, Ytrue := mat.NewDense(len(pred), 1, pred), mat.NewDense(len(tru), 1, tru)
	return metrics.AccuracyScore(Ytrue, Ypred, normalize, sampleWeight)

}

// call command line executable
func prepareDataset(dataset string){
	cmd :=exec.Command("./extract_splitDataset.sh", dataset)
	cmd.Run()
}

func Mean(array []float64)float64{
	sum := 0.0
    for _, num := range array {
        sum += num
    }

    // Calculate the mean
    mean := sum / float64(len(array))
	return mean
}

func outlierRatio(lab []int) float64{
	count :=0 
	for _,v := range lab{
		if v == 1{
			count++
		}
	}
	anomaly := float64(count)/float64(len(lab))
	return anomaly
}

func findMaxIndex(arr []float64)int{

	maxIndex := 0 

    for i := 1; i < len(arr); i++ {
     
        if arr[i] > arr[maxIndex] {
            maxIndex = i
        }
    }

    return maxIndex 
}


func BuildLabelsArray(lendata int, lenzerodata int)[]int{


labeldata := make([]int, lendata)
			for i:=0; i<lendata; i++{
				if i < lenzerodata{
				labeldata[i] = 0
				}else{
				labeldata[i] = 1
			}
		}
		return labeldata
}

func findBestConfig(themap map[config][]float64)*config{
	
	highest_mean := 0.0
	var bestconf *config
	for conf,f1s := range themap{
		mean := Mean(f1s)	
		
		if mean > highest_mean{
			highest_mean = mean
			bestconf = &config{
                treeNum:       conf.treeNum,
                subsamplesize: conf.subsamplesize,
			}
		}
	}
	fmt.Println("Highest mean is: ",highest_mean)
	return bestconf
}





# TRAIN TEST FUNCTION

In [3]:
//Isolation Forest algorithm

func isoForestTrain_Test(training_data [][]float64 ,testing_data [][]float64 , treesNumber int, subsampleSize int, outliers float64) *iforest.Forest{
//model initialization
forest := iforest.NewForest(treesNumber, subsampleSize, outliers)

//training stage - creating trees
forest.Train(training_data)

//testing stage - finding anomalies 
forest.Test(testing_data)

//threshold := forest.AnomalyBound
//labels := forest.Labels

//fmt.Println("threshold is",threshold)

return forest;
}


# Split Data, K-Fold Utils

In [4]:


func SplitDataset (dataset [][]float64, labels[]int, splitRatio float64)(map[int][][]float64,map[int][][]float64,float64){

	myMap := make(map[int][][]float64)
	TrainingIndex_Data := make(map[int][][]float64)
	TestingIndex_Data := make(map[int][][]float64)
	count:=0
	for i,instance:= range dataset{
		if labels[i] == 0{
		myMap[0] = append(myMap[0], instance)
		}else{
		myMap[1] = append(myMap[1], instance)
		count++
		}
	}
	ShuffleUnit(myMap)
	split0:= int(math.Round(splitRatio * float64(len(myMap[0]))))
	split1:= int(math.Round(splitRatio * float64(len(myMap[1]))))

	TrainingIndex_Data[0] = myMap[0][:split0]
	TrainingIndex_Data[1] = myMap[1][:split1]
	TestingIndex_Data[0] = myMap[0][split0:]
	TestingIndex_Data[1] = myMap[1][split1:] 

	anomaly := float64(count)/float64(len(labels))
	return  TrainingIndex_Data, TestingIndex_Data, anomaly
}

func Shuffle(testing_set [][]float64, training_set [][]float64, label_set []int){

	rand.Shuffle(len(training_set), func(i, j int) {
		training_set[i], training_set[j] = training_set[j], training_set[i]
	})
	rand.Shuffle(len(testing_set), func(i, j int) {
		testing_set[i], testing_set[j] = testing_set[j], testing_set[i]
		label_set [i], label_set[j] = label_set[j], label_set[i]
	})
}

func ShuffleUnit(data_label map[int][][]float64){

	rand.Shuffle(len(data_label[0]), func(i, j int) {
		data_label[0][i], data_label[0][j] = data_label[0][j], data_label[0][i]
	})
	rand.Shuffle(len(data_label[1]), func(i, j int) {
		data_label[1][i], data_label[1][j] = data_label[1][j], data_label[1][i]
	})
}

func StratifiedKFold(K int, data_label map[int][][]float64)(ch chan Split){


	foldSizezeros:= int(math.Round(float64((len(data_label[0])/K))))
	foldSizeones:= int(math.Round((float64(len(data_label[1])/K))))

	ch = make(chan Split)
	
	go func(){
		
		var sp *Split
		for i:=0; i<K; i++{
			startIndex0 := i * foldSizezeros
			endIndex0 := (i + 1) * foldSizezeros
			startIndex1 := i * foldSizeones
			endIndex1 := (i + 1) * foldSizeones

			if i == K-1 {
				endIndex0 = len(data_label[0])
				endIndex1 = len(data_label[1])
			}
			trainData0 := make([][]float64, 0)
			trainData1 := make([][]float64, 0)

			// Merge all data except the test data into the training data
			trainData0 = append(trainData0, data_label[0][:startIndex0]...)
			trainData0 = append(trainData0, data_label[0][endIndex0:]...)
			trainData1 = append(trainData1, data_label[1][:startIndex1]...)
			trainData1 = append(trainData1, data_label[1][endIndex1:]...)
			
			trainData := Merge(trainData0,trainData1)
			

			validData0:=data_label[0][startIndex0:endIndex0]
			validData1:=data_label[1][startIndex1:endIndex1]
			
			validData := Merge(validData0, validData1)

			validDatalen:= len(validData)
		
		    labeldata:= BuildLabelsArray(validDatalen,len(validData0))
			
			Shuffle(validData,trainData,labeldata)
			
			sp = &Split{
				TrainData: trainData,
				ValidateData:  validData,
				LabelData: labeldata,
				//trainLabelData: trainlabeldata,
			}
			ch <- *sp
		}
		close(ch)	
	}()
	return ch
}

func Merge(data1 [][]float64, data2 [][]float64)[][]float64{

	concatenated:=make([][]float64, len(data1)+len(data2))
					copy(concatenated, data1)
    				copy(concatenated[len(data1):], data2)
	return concatenated
}
 

# Grid Search Cross Validation

In [7]:

func GridSearchCV(data map[int][][]float64,treenummax int,subsamplmax int,anomaly float64) *config{

	ConfF1s := make(map[config][]float64,0)
	var conf config
	i := 1
	
	for s := range StratifiedKFold(5,data){	
		treenumStep := 10
		fmt.Println("Entering Iteration num ",i)
		for tr_n :=10 ;tr_n <=treenummax; tr_n+= treenumStep{
			for sss := 50; sss<=subsamplmax; sss+= 50{
				conf = config{
					treeNum:tr_n,
					subsamplesize:sss,
				}
				forest:=isoForestTrain_Test(s.TrainData, s.ValidateData,tr_n,sss,anomaly)
				f1Score := F1Score(forest.Labels,s.LabelData)
				ConfF1s[conf] = append(ConfF1s[conf],f1Score)
			}
			if tr_n == 100{
				treenumStep = 100
			}else if tr_n == 1000{
				treenumStep = 500
			}
		}
		i++
	}
	bestConfig:=findBestConfig(ConfF1s)
	fmt.Println(ConfF1s)
	
	return bestConfig
}

# Main Pipeline

In [ ]:
%%
//load from files the pr 
data:=loadData("dataset.csv")
labels:=LoadLabels("labels.csv")

training_data , testing_data , anomaly :=  SplitDataset(data,labels,0.8)
fmt.Println("anomaly ratio is: ",anomaly)

configuration := GridSearchCV(training_data,2000,350,anomaly) //returns the Best configuration {treeNum}{SubsamplingSize}


//train with the best conf
fmt.Println("Training Model with parameters... ",configuration)
forestt := isoForestTrain_Test(data,data,configuration.treeNum,configuration.subsamplesize,anomaly)

//prepare the testing data for predict
testing := Merge(testing_data[0], testing_data[1])

testDatalen := len(testing)
testData0len :=len(testing_data[0])

labeldata:=BuildLabelsArray(testDatalen,testData0len)


predictlabels,_,_ :=forestt.Predict(testing)
fmt.Println("Predict Accuracy is...",AccuracyScore(predictlabels,labeldata))
fmt.Println("Predict F1 is...",F1Score(predictlabels,labeldata))





Highest mean is:  0.7124137435571408

Training Model with parameters...  &{90 150}

Predict Accuracy is... 0.9206623016557541

Predict F1 is... 0.7017300174643211

# Experiment with the best configuration

In [21]:
%%
data:=loadData("dataset.csv")
labels:=LoadLabels("labels.csv")

forestt := isoForestTrain_Test(data,data,90,150,0.07193103596551799)
predict_labels := forestt.Labels
fmt.Println("Predict Accuracy is...",AccuracyScore(predict_labels,labels))
fmt.Println("Predict F1 is...",F1Score(predict_labels,labels))

9.175657275024252
Predict Accuracy is... 0.9222264611132306
Predict F1 is... 0.7087427817347027


# Unit Testing

In [ ]:

func dataLabelGenerator(numberofinstances int)([][]float64 ,[]int){
	data := make([][]float64,0)
	labels := make([]int,0)
	count:=0
	for j:=0; j<numberofinstances; j++{
		label := rand.Intn(2)
		labels = append(labels,label)
		if label == 0 {
			data = append(data,[]float64{0.0, 0.0})
		}else{
			data= append(data,[]float64{1.1,1.1})
			count++
		}	
	} 

	
	return data,labels
}

func TestSplitDataset(ratio float64)(map[int][][]float64,map[int][][]float64, float64){

	data,labels := dataLabelGenerator(532)

	data_label_train,data_label_test,anomaly:=SplitDataset(data,labels,ratio)
	trainlen0:=len(data_label_train[0])
	trainlen1:=len(data_label_train[1])
    testlen0:=len(data_label_test[0])
	testlen1:=len(data_label_test[1]) 
 
    trainlen := trainlen0+trainlen1
	testlen := testlen0+testlen1 
	right_train_len := int(float64(len(data))*ratio)
	right_test_len := int(float64(len(data))*(1-ratio))
	fmt.Println("SPLITDATASET METRICSSSSSSS")
	fmt.Println("train length",trainlen)
	fmt.Println("what train length should be",right_train_len)
	fmt.Println("test length",testlen)
	fmt.Println("what train test should be",right_test_len)

	traininganomaly := float64(trainlen1)/float64(trainlen)
	testinganomaly := float64(testlen1)/float64(testlen)

	fmt.Println("real anomaly ratio",anomaly)
	fmt.Println("training set anomaly ratio",traininganomaly)
	fmt.Println("testing set anomaly ratio",testinganomaly)

	return 	data_label_train,data_label_test,anomaly

}

func TestStratifiedKFold(){

	data_label_train,_ ,anomaly:= TestSplitDataset(0.8)
	fmt.Println("KFOLD METRICSSSSSSS")
	for s := range StratifiedKFold(5,data_label_train){
		traindatalen:=len(s.TrainData)
		testdatalen:=len(s.ValidateData)
		validatedatalen:=len(s.ValidateData)
		Kfoldtotallength:= traindatalen+validatedatalen
		Initiallen:= len(data_label_train[0])+len(data_label_train[1])
		fmt.Println("validate data length",validatedatalen)
		fmt.Println("traindata length",traindatalen)
		fmt.Println("total length length", Kfoldtotallength)
		fmt.Println("should be the same as", Initiallen)
		count:=0
		//count1:=0
		for _,v:= range s.LabelData{
			if v==1{
				count++
			} 
		}
		/* for _,v:= range s.trainLabelData{
			if v==1{
				count1++
			} 
		} */
		testinganomaly:= float64(count)/float64(testdatalen)
		//traininganomaly:= float64(count1)/float64(traindatalen)

		fmt.Println("real anomaly ratio",anomaly)
		//fmt.Println("Kfold training set anomaly ratio",traininganomaly)
		fmt.Println("Kfold testing set anomaly ratio",testinganomaly)



}

}

%%
///test best config
myMap := make(map[config][]float64)
myMap[config{10, 50}] = []float64{0.5663299756015053, 0.5784793547873575, 0.602391343317249, 0.6939057554846884, 0.5689573648735637}
myMap[config{10, 100}] = []float64{0.7276119962614284, 0.7033651350563244, 0.6943534857604245, 0.7365462700324026, 0.7053684071844848}
myMap[config{20, 50}] = []float64{0.5864679952729727, 0.6908322825748243, 0.6938493643294152, 0.6978173332046268, 0.6560886870122624}
myMap[config{20, 100}] = []float64{0.7354727967361132, 0.7005686557080835, 0.715626945402174, 0.7171468309980348, 0.6951911213811953}

bestConfig:=findBestConfig(myMap)
fmt.Println(bestConfig)


///test Kfold

TestStratifiedKFold()

# Junk Code :( 

In [ ]:

func KFold(K int, datasetlen int)(ch chan Split){

	Kpoint :=  int(math.Ceil(float64(datasetlen)/float64(K)))
	Kpoints:=[]int{0}

	for j:=1; j< K; j++{
		if Kpoints[j-1] + Kpoint >= datasetlen {
			break
		} 
		Kpoints = append(Kpoints, Kpoints[j-1] + Kpoint)
		
	}
	Kpoints = append(Kpoints, datasetlen)

    Indexes:=make([]int, datasetlen)
	for i:=0; i<datasetlen; i++{
		Indexes[i]=i
		
	} 

	ch = make(chan Split)
	
	go func(){
		var sp *Split
		for testsplit:=0; testsplit<K; testsplit++{
			if testsplit == 0{
				sp = &Split{
					TestIndex: Indexes[:Kpoints[1]],
					TrainIndex:  Indexes[Kpoints[1]:],				
				}

				}else if testsplit == K-1{
					sp = &Split{
					TestIndex: Indexes[Kpoints[K-1]:],
					TrainIndex:  Indexes[:Kpoints[K-1]],
					}
				}else{
					before:= Indexes[:Kpoints[testsplit]]
					after:= Indexes[Kpoints[testsplit+1]:]
					concatenated:=make([]int, len(before)+len(after))
					copy(concatenated, before)
    				copy(concatenated[len(before):], after)

					sp = &Split{
					TestIndex: Indexes[Kpoints[testsplit]:Kpoints[testsplit+1]],
					TrainIndex:  concatenated,
				}
			}
			ch <-*sp
		}
		close(ch)
	}()
	return ch

}
func getElements(indexTrain []int ,indexTest []int ,dataArray [][]float64, labelsArray []int)([][]float64,[][]float64, []int) {
	dataTrain := make([][]float64, len(indexTrain))
	dataTest := make([][]float64, len(indexTest))
	labels := make([]int, len(indexTest))
	
	for i, index := range indexTrain {
		dataTrain[i] = dataArray[index]
	}
	for i, index := range indexTest {
		dataTest[i] = dataArray[index]
		labels[i] = labelsArray[index]

	}
	return dataTrain,dataTest,labels
}

 func CrossValidation(dataset string){
	
	i:=1
	data:=loadData(dataset)
	labels:=LoadLabels("training_labels.csv")
	for s := range KFold(5,len(data)){
		fmt.Println("%Iteration N.",i)
		i++
		traindata,testdata,labeldata := getElements(s.TrainIndex,s.TestIndex,data,labels)
		forest:=isoForestTrain_Test(traindata, testdata,100,1000,0.8)
		AccuracyScore(forest.Labels,labeldata)
		F1Score(forest.Labels,labeldata)	
	}

}
%%
/* data := loadData("training.csv")
fmt.Println(data)
//datasetlen:=len(data)
for s := range KFold(5,data){
	fmt.Println(s.TrainData)
	
} */

cache.ResetKey("my_forest")
var forest = cache.Cache("my_forest", func() * iforest.Forest {return isoForestTrain_Test(data,data,configuration.treeNum,configuration.subsamplesize,outlier)})

///////////////////////


type config struct{
	treeNum int
	subsamplesize int
}

func CrossValidation(data map[int][][]float64,treenummax int,subsamplmax int,anomaly float64) *config{

	meanF1Arr := make([]float64,0)
	configArr := make([]config,0)
	treenumStep := 10

	for tr_n :=10 ;tr_n <=treenummax; tr_n+= treenumStep{
		for sss := 50; sss<=subsamplmax; sss+= 50{
			var conf *config
			fmt.Println("Tree number = ",tr_n)
			fmt.Println("Subsamplingsize = ",sss)
			//Acc_arr:=make([]float64,0)
			F1_arr:=make([]float64,0)
			for s := range StratifiedKFold(5,data){
					forest:=isoForestTrain_Test(s.TrainData, s.ValidateData,tr_n,sss,anomaly)
					//Acc_arr = append(Acc_arr,AccuracyScore(forest.Labels,s.LabelData))
					F1_arr = append(F1_arr,F1Score(forest.Labels,s.LabelData))
				}
					
			mean_f1:= Mean(F1_arr)
			fmt.Println("Configuration Mean F1 Score is ",mean_f1)
		    meanF1Arr = append(meanF1Arr,mean_f1)

		    conf = &config{
				treeNum:tr_n,
				subsamplesize:sss,
			}

			configArr = append(configArr,*conf)
		}
		if tr_n == 100{
			treenumStep = 100
		}else if tr_n == 1000{
			treenumStep = 500
		}
	}
	maxIndex := findMaxIndex(meanF1Arr)
	bestConfig := configArr[maxIndex]
	return &bestConfig
}